# Imports and loading

## loading

In [ ]:
# System
import os
import gc
import subprocess
import keras
from keras.preprocessing import image

from PIL import Image

# Linear algebra and ML tools
import numpy as np
import scipy as scp
import pandas as pd

# Data Visualisation
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid", palette='husl')
plt.rcParams['legend.framealpha'] = 1
plt.rcParams['legend.frameon'] = True
plt.rcParams['legend.edgecolor'] = 'k'
plt.rcParams['legend.fancybox'] = False
plt.rcParams['legend.facecolor'] = 'w'
plt.rcParams['legend.handlelength'] = 1
plt.rcParams['legend.handleheight'] = 1.125
plt.rcParams['figure.figsize'] = (16, 9)

INPUT_PATH = '../input/'
TRAIN_PATH = INPUT_PATH + 'train/'
%load_ext memory_profiler

label_mapping = {
    'class_0': 'Nucleoplasm',
    'class_1': 'Nuclear membrane',
    'class_2': 'Nucleoli',
    'class_3': 'Nucleoli fibrillar center',
    'class_4': 'Nuclear speckles',
    'class_5': 'Nuclear bodies',
    'class_6': 'Endoplasmic reticulum',
    'class_7': 'Golgi apparatus',
    'class_8': 'Peroxisomes',
    'class_9': 'Endosomes',
    'class_10': 'Lysosomes',
    'class_11': 'Intermediate filaments',
    'class_12': 'Actin filaments',
    'class_13': 'Focal adhesion sites',
    'class_14': 'Microtubules',
    'class_15': 'Microtubule ends',
    'class_16': 'Cytokinetic bridge',
    'class_17': 'Mitotic spindle',
    'class_18': 'Microtubule organizing center',
    'class_19': 'Centrosome',
    'class_20': 'Lipid droplets',
    'class_21': 'Plasma membrane',
    'class_22': 'Cell junctions',
    'class_23': 'Mitochondria',
    'class_24': 'Aggresome',
    'class_25': 'Cytosol',
    'class_26': 'Cytoplasmic bodies',
    'class_27': 'Rods & rings',
}

## loading train labels and filenames

In [ ]:
n_classes = len(label_mapping)
train_labels = pd.read_csv(INPUT_PATH + 'train.csv')
train_labels['Target'] = train_labels['Target'].apply(lambda x: list(map(int, x.split(' '))))

def vectorize(arr, size=n_classes):
    res = np.zeros(size, dtype=int)
    res[arr] = 1
    return res

train_labels['target_v'] = train_labels['Target'].apply(vectorize)
train_labels[list(label_mapping.keys())] = pd.DataFrame(train_labels['target_v'].values.tolist(), index=train_labels.index)
train_labels.head(5)

### green channel only

In [ ]:
# working with green channel only
train_labels['Id'] = train_labels['Id'] + '_green.png'
train_labels.head()

## class distribution

In [ ]:
class_count = train_labels['target_v'].sum()
pd.Series(class_count).plot('bar', width=0.8)

## Nb labels distribution

In [ ]:
label_count = train_labels['target_v'].apply(sum)
label_count.value_counts().plot('bar', width=0.8)

## StratifiedShuffleSplit


### coalesce target to stratify on under-represented classes

In [ ]:
def coalesce(arr):
    return arr[np.argmin(np.array(class_count)[arr])]
    
train_labels['y_coal'] = train_labels['Target'].apply(coalesce)
train_labels.head(5)

In [ ]:
train_labels['y_coal'].value_counts().sort_index().plot('bar', width=0.8)

### StratifiedShuffleSplit

### if train/test/val

In [ ]:
# from sklearn.model_selection import StratifiedShuffleSplit

# sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
# for train_index, test_index in sss.split(np.zeros(len(train_labels)), train_labels['y_coal']):
#     df_train = train_labels.iloc[train_index]
#     df_test = train_labels.iloc[test_index]

# for train_index, val_index in sss.split(np.zeros(len(df_train)), df_train['y_coal']):
#     df_val = df_train.iloc[val_index]
#     df_train = df_train.iloc[train_index]

# # reset_index
# df_train.reset_index(drop=True, inplace=True)
# df_val.reset_index(drop=True, inplace=True)
# df_test.reset_index(drop=True, inplace=True)

# print('nb_train: {} / class_count: {}'.format(len(df_train), df_train.target_v.sum()))
# print('nb_val: {} / class_count: {}'.format(len(df_val), df_val.target_v.sum()))
# print('nb_test: {} / class_prop: {}'.format(len(df_test), df_test.target_v.sum()))

### if train/val only

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
for train_index, val_index in sss.split(np.zeros(len(train_labels)), train_labels['y_coal']):
    df_train = train_labels.iloc[train_index]
    df_val = train_labels.iloc[val_index]

# reset_index
df_train.reset_index(drop=True, inplace=True)
df_val.reset_index(drop=True, inplace=True)

print('nb_train: {} / class_count: {}'.format(len(df_train), df_train.target_v.sum()))
print('nb_val: {} / class_count: {}'.format(len(df_val), df_val.target_v.sum()))

In [ ]:
# some gc collection
del train_labels
import gc
gc.enable()
gc.collect()

# Running CNN

## utils

In [ ]:
from keras import backend as K
import tensorflow as tf

def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)

## DataGenerator

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

batch_size_train = 64
batch_size_val = 64
WIDTH = 256
HEIGHT = 256
channels = 1

datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_dataframe(dataframe=df_train, directory=TRAIN_PATH, x_col="Id", y_col=list(label_mapping.keys()), color_mode='grayscale',
                                              class_mode='other', target_size=(WIDTH,HEIGHT), batch_size=batch_size_train)
val_generator = datagen.flow_from_dataframe(dataframe=df_val, directory=TRAIN_PATH, x_col="Id", y_col=list(label_mapping.keys()), color_mode='grayscale',
                                            class_mode="other", target_size=(WIDTH,HEIGHT), batch_size=batch_size_val)

## Model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D

filters0 = 8
nb_dense = 64

model = Sequential()

model.add(Conv2D(filters0, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(WIDTH, HEIGHT, channels)))
model.add(Conv2D(filters0, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(WIDTH, HEIGHT, channels)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters0 * 2, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(Conv2D(filters0 * 2, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters0 * 4, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(Conv2D(filters0 * 4, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters0 * 4, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(Conv2D(filters0 * 4, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters0 * 8, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(Conv2D(filters0 * 8, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(Conv2D(filters0 * 8, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters0 * 16, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(Conv2D(filters0 * 16, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(Conv2D(filters0 * 16, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(nb_dense, activation='relu'))
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

In [ ]:
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

STEP_SIZE_TRAIN = train_generator.n // train_generator.batch_size
STEP_SIZE_VALID = val_generator.n // val_generator.batch_size

epochs = 100
learning_rate = 0.0001
callback= EarlyStopping(monitor='val_loss', patience=40)

model.compile(loss=f1_loss,
              optimizer=Adam(lr=learning_rate),
              metrics=[f1])

output = model.fit_generator(
    train_generator,
    steps_per_epoch=STEP_SIZE_TRAIN,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=STEP_SIZE_VALID,
    callbacks=[callback],
    use_multiprocessing=True,
    )

In [ ]:
model.save_weights('multilabel_cnn_green.h5')

## Result analysis

In [ ]:
# Plotting the performance of our network
plt.plot(output.epoch, output.history['loss'], label='train')
plt.plot(output.epoch, output.history['val_loss'], label='val')
plt.title('Training and validation performance')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend()

# Test performance

In [ ]:
# y_predict_proba = model.predict()
# f1_score(y_train, y_predict_proba.round(), average='macro')

In [ ]:
# diff = y_val != y_predict_proba.round()
# x_test_errors = X_val[diff]
# y_test_errors = y_val_class[diff]
# y_predict_errors = y_predict[diff]
# y_predict_proba_errors = y_predict_proba[diff]

# print('{} errors out of {}'.format(len(x_val_errors), len(X_val)))

# index = np.random.randint(len(y_predict_errors))

# print("Correct label is: ", y_val_errors[index])
# print("Predicted label is: ", y_predict_errors[index])
# print("Probabilities: ", y_predict_proba_errors[index])
# plt.imshow(np.squeeze(x_test_errors[index]))